In [1]:
!pip install -qU langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
import inspect
from getpass import getpass
from langchain import OpenAI
from langchain import LLMChain , ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import gellt_openai_callback
import tiktoken

In [3]:
OPENAI_API_KEY = getpass()

··········


In [4]:
llm=OpenAI(temperature=0 , model_name="text-davinci-003", openai_api_key=OPENAI_API_KEY)


In [7]:
def count_tokens(chain , query):
  with get_openai_callback() as cb:
    result=chain.run(query)
    print(f'Spent a total of {cb.total_tokens} tokens')

  return result


**ConversationBufferMemory** :
The ConversationBufferMemory does just what its name suggests: it keeps a buffer of the previous conversation excerpts as part of the context in the prompt.

In [6]:
conversation_buf=ConversationChain(llm=llm,memory=ConversationBufferMemory())

In [8]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': " Good morning! It's a beautiful day today, isn't it? How can I help you?"}

In [9]:
count_tokens(
    conversation_buf,
    "I want to know about integrating Artificial Intelligence in daily lives"
)

Spent a total of 206 tokens


" Sure! AI is becoming increasingly popular in our daily lives. It's being used in a variety of ways, from helping us with our daily tasks to providing us with personalized recommendations. For example, AI can be used to automate mundane tasks, such as scheduling appointments or managing emails. AI can also be used to provide personalized recommendations, such as suggesting movies or restaurants based on your preferences. AI can even be used to help us make better decisions, such as providing us with financial advice or helping us make better health choices."

In [10]:
  count_tokens(
    conversation_buf,
    "Which data sources can give more context for AI to train on"
)

Spent a total of 322 tokens


' AI typically uses data from a variety of sources to train its algorithms. This data can come from a variety of sources, such as public databases, private databases, or even user-generated data. For example, AI can use data from social media platforms to learn about user preferences and behaviors. AI can also use data from sensors, such as cameras or microphones, to learn about the environment. AI can also use data from medical records or financial records to learn about health or financial trends.'

In [12]:
count_tokens(
    conversation_buf,
    "What is my aim again?"
)

Spent a total of 348 tokens


' Your aim is to learn about integrating Artificial Intelligence into daily lives.'

In [13]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI:  Good morning! It's a beautiful day today, isn't it? How can I help you?
Human: I want to know about integrating Artificial Intelligence in daily lives
AI:  Sure! AI is becoming increasingly popular in our daily lives. It's being used in a variety of ways, from helping us with our daily tasks to providing us with personalized recommendations. For example, AI can be used to automate mundane tasks, such as scheduling appointments or managing emails. AI can also be used to provide personalized recommendations, such as suggesting movies or restaurants based on your preferences. AI can even be used to help us make better decisions, such as providing us with financial advice or helping us make better health choices.
Human: Which data sources can give more context for AI to train on
AI:  AI typically uses data from a variety of sources to train its algorithms. This data can come from a variety of sources, such as public databases, private databases, or even user-ge

**ConversationSummaryMemory**  - the conversation summary memory keeps the previous pieces of conversation in a summarized form, where the summarization is performed by an LLM.

In [14]:
conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

In [15]:
count_tokens(
    conversation_sum,
    "Good morning AI!"
)

Spent a total of 290 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [16]:
count_tokens(
    conversation_sum,
    "I want to know about integrating Artificial Intelligence in daily lives"
)

Spent a total of 554 tokens


" Sure, I'd be happy to help. Artificial Intelligence is becoming increasingly popular in our daily lives. It is being used in many different ways, such as in healthcare, transportation, and even in our homes. AI can be used to automate mundane tasks, provide personalized recommendations, and even help us make decisions. AI can also be used to improve customer service, increase efficiency, and reduce costs."

In [17]:
  count_tokens(
    conversation_sum,
    "Which data sources can give more context for AI to train on"
)

Spent a total of 821 tokens


' AI can be trained on a variety of data sources, including structured data from databases, unstructured data from text documents, audio, and video, as well as real-time data from sensors and other sources. AI can also be trained on historical data, such as customer records, financial data, and other data sets. Additionally, AI can be trained on data from external sources, such as social media, news articles, and other public sources.'

In [18]:
count_tokens(
    conversation_sum,
    "What is my aim again?"
)

Spent a total of 868 tokens


' It looks like you were asking about integrating Artificial Intelligence into daily life. Is that correct?'

In **ConversationBufferWindowMemory**  we will be keeping a few of the last interactions in our memory but we will intentionally drop the oldest ones - short-term memory if you'd like. Here the aggregate token count and the per-call token count will drop noticeably. We will control this window with the k parameter.

In [19]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

In [20]:
count_tokens(
    conversation_bufw,
    "Good morning AI!"
)

Spent a total of 85 tokens


" Good morning! It's a beautiful day today, isn't it? How can I help you?"

In [22]:
count_tokens(
    conversation_bufw,
    "I want to know about integrating Artificial Intelligence in daily lives"
)

Spent a total of 203 tokens


" Sure! AI is becoming increasingly popular in our daily lives. It's being used in a variety of ways, from helping us with our daily tasks to providing us with personalized recommendations. For example, AI can be used to automate mundane tasks, such as scheduling appointments or managing emails. AI can also be used to provide personalized recommendations, such as suggesting movies or restaurants based on your preferences. AI can even be used to help us make decisions, such as providing financial advice or helping us choose the best products."

In [23]:
  count_tokens(
    conversation_bufw,
    "Which data sources can give more context for AI to train on"
)

Spent a total of 293 tokens


' AI typically uses data from a variety of sources to train its algorithms. This data can come from a variety of sources, such as public databases, private databases, or even user-generated data. For example, AI can use data from social media platforms to learn about user preferences and behaviors. AI can also use data from sensors, such as cameras or microphones, to learn about the environment. AI can also use data from other sources, such as weather data or traffic data, to make predictions about the future.'

In [24]:
count_tokens(
    conversation_bufw,
    "What is my aim again?"
)

Spent a total of 213 tokens


' Your aim is to use data from a variety of sources to train AI algorithms so that they can make more accurate predictions and decisions.'

We can see, it  'forgot' what we talked about in the first interaction. Let's see what it 'remembers'. Given that we set k to be 1, we would expect it remembers only the last interaction.

In [25]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']
print(bufw_history)

Human: What is my aim again?
AI:  Your aim is to use data from a variety of sources to train AI algorithms so that they can make more accurate predictions and decisions.


In [27]:
tokenizer = tiktoken.encoding_for_model('text-davinci-003')
print(
    f'Buffer memory conversation length: {len(tokenizer.encode(conversation_buf.memory.buffer))}\n'
    f'Summary memory conversation length: {len(tokenizer.encode(conversation_sum.memory.buffer))}\n'
    f'Buffer window memory conversation length: {len(tokenizer.encode(bufw_history))}'
)

Buffer memory conversation length: 294
Summary memory conversation length: 218
Buffer window memory conversation length: 38


More memory types! <br><br>

**ConversationSummaryBufferMemory**
: the conversation summary memory keeps a summary of the earliest pieces of conversation while retaining a raw recollection of the latest interactions.
<br><br>
**ConversationKnowledgeGraphMemory**
This is a super cool memory type that was introduced just recently. It is based on the concept of a knowledge graph which recognizes different entities and connects them in pairs with a predicate resulting in (subject, predicate, object) triplets.





**The conversation knowledge graph memory** keeps a knowledge graph of all the entities that have been mentioned in the interactions together with their semantic relationships.

In [28]:
conversation_kg = ConversationChain(
    llm=llm,
    memory=ConversationKGMemory(llm=llm)
)

In [29]:
count_tokens(
    conversation_kg,
    "I am Saahil , I am an engineer and I like to study AI and play cricket"
)

Spent a total of 1234 tokens


" Hi Saahil, it's nice to meet you. I'm an AI and I'm interested in learning more about AI and cricket. Can you tell me more about your experience with AI and cricket?"

The memory keeps a knowledge graph of everything it learned so far.

In [30]:
conversation_kg.memory.kg.get_triples()

[('Saahil', 'engineer', 'is'),
 ('Saahil', 'AI', 'likes to study'),
 ('Saahil', 'cricket', 'likes to play')]